In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
#using a relative path
file_path = "../data/AmesHousing.csv"
df = pd.read_csv(file_path)

df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [ ]:
X = df[['Overall Qual', 'Gr Liv Area', 'Garage Cars']].fillna(0)
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)

print("R²:", round(r2_score(y_test, y_pred), 3))
print("Coefficients:", dict(zip(X.columns, model.coef_)))
print("Intercept:", round(model.intercept_, 2))


R²: 0.76
Coefficients: {'Overall Qual': np.float64(27600.794042200378), 'Gr Liv Area': np.float64(50.23646269367018), 'Garage Cars': np.float64(20017.12258659879)}
Intercept: -98749.69


#### The regression shows a strong linear relationship: home sale price increases by about $27.6 k per quality point, $50 per square foot, and $20 k per garage space, explaining roughly 76% of price variation.